In [14]:

# conda remove --name SCENIC --all
# conda create --name SCENIC python=3.11
# conda activate SCENIC
# pip install ipykernel
# python -m ipykernel install --user --name SCENIC --display-name "Python SCENIC"
# git clone https://github.com/aertslab/scenicplus
# cd scenicplus
# git checkout development # this will install the developing version. It is not necessary
# conda install conda-forge::rust
# conda install conda-forge::gcc
# conda install conda-forge::python-devtools
# conda install -c conda-forge::gxx
# pip install libpython
# pip install -e . --log pip.log
# pip install matplotlib==3.6.3
# pip install -U ipywidgets
# pip install pyranges==0.0.129 #it complains but it works
# pip install numba==0.58.1 # this solves:   if low_memory: nn_descent_internal_low_memory_parallel(
# pip install  umap-learn==0.5.4 # this solves:   if low_memory: nn_descent_internal_low_memory_parallel(
# delete the row:
#if verbose:
#print("\t", n + 1, " / ", n_iters)
#in pynndescent_.py with function nn_descent_internal_low_memory_parallel and nn_descent_internal_high_memory_parallel can avoid this error.
#On the other, this error only occur when cells above 5000.

# Options wasnt working and i editted the .py that was complaning files 
import warnings
warnings.simplefilter(action='ignore')
import os

#os.environ['OPENBLAS_NUM_THREADS'] = '1' # helps with error: OpenBLAS blas_thread_init: RLIMIT_NPROC 4096 current, 1540503 max
#OpenBLAS blas_thread_init: pthread_create failed for thread 39 of 40: Resource temporarily unavailable
#OpenBLAS blas_thread_init: RLIMIT_NPROC 4096 current, 1540503 max
#Resource temporarily unavailable (src/thread.cpp:269)


In [16]:
import pycisTopic
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
from pycisTopic.iterative_peak_calling import *
import pandas as pd
import scanpy
import requests
import pyranges 
import anndata
import pickle
import pyarrow 
import re
import dill



In [18]:
import  pyranges
pyranges.__version__
#'0.0.127'

'0.0.127'

In [20]:
import pynndescent
pynndescent.__version__
#'0.5.12'

'0.5.12'

In [22]:
import umap
umap.__version__
#'0.5.4'

'0.5.4'

In [24]:
import numba
numba.__version__
#'0.58.1'




'0.58.1'

In [26]:
#os.environ[“HDF5_USE_FILE_LOCKING”] = “FALSE”

#HDF5_USE_FILE_LOCKING= False



def CountFrequency(my_list):
 
    # Creating an empty dictionary
    freq = {}
    for items in my_list:
        freq[items] = my_list.count(items)
 
    for key, value in freq.items():
        print("% s : % d" % (key, value))
 

outDir="SCENIC_results_MARCH2024/"
tmpDir="/tmp/"
#ray.shutdown()




In [28]:
outDir

'SCENIC_results_MARCH2024/'

In [30]:
#We now have completed all the steps necessary for starting the SCENIC+ analysis 😅.

#In particalular, we have

#preprocessed the scRNA-seq side of the data, selecting high quality cells and annotation these cells.

#preprocessed the scATAC-seq side of the data, selecting high quality cells, performing topic modeling and identifying candidate enhacer regions.

#looked for enriched motifs in candidate enhancer regions.

#In the next section we will combine all these analysis and run SCENIC+



In [32]:
pwd

'/fs03/df22/perlaza/margo.montandon/SCENIC_MARCH2024'

In [34]:


# Set stderr to null to avoid strange messages from ray
_stderr = sys.stderr
null = open(os.devnull,'wb')


cistopic_obj = dill.load(open(os.path.join(outDir, 'cisTopicObject_noDBL.pkl'), 'rb'))
cistopic_obj

In [36]:

f=open('SCENIC_results_MARCH2024/motifs/menr.pkl','rb')
f



<_io.BufferedReader name='SCENIC_results_MARCH2024/motifs/menr.pkl'>

In [38]:
pyranges.pyranges_main

<module 'pyranges.pyranges_main' from '/home/lper0012/.conda/envs/SCENIC/lib/python3.11/site-packages/pyranges/pyranges_main.py'>

In [40]:
menr = dill.load(f)



In [42]:
menr

{'CTX_Topics_All': {'Topic1': <pycistarget.motif_enrichment_cistarget.cisTarget at 0x7f83186d46d0>,
  'Topic2': <pycistarget.motif_enrichment_cistarget.cisTarget at 0x7f83454aa410>,
  'Topic3': <pycistarget.motif_enrichment_cistarget.cisTarget at 0x7f832bbf0650>,
  'Topic4': <pycistarget.motif_enrichment_cistarget.cisTarget at 0x7f8314641ed0>,
  'Topic5': <pycistarget.motif_enrichment_cistarget.cisTarget at 0x7f8312939150>,
  'Topic6': <pycistarget.motif_enrichment_cistarget.cisTarget at 0x7f830dd7bd50>,
  'Topic7': <pycistarget.motif_enrichment_cistarget.cisTarget at 0x7f830665b3d0>,
  'Topic8': <pycistarget.motif_enrichment_cistarget.cisTarget at 0x7f82ff833a90>,
  'Topic9': <pycistarget.motif_enrichment_cistarget.cisTarget at 0x7f82f6318350>,
  'Topic10': <pycistarget.motif_enrichment_cistarget.cisTarget at 0x7f82eb35f850>,
  'Topic11': <pycistarget.motif_enrichment_cistarget.cisTarget at 0x7f8318ad9c10>,
  'Topic12': <pycistarget.motif_enrichment_cistarget.cisTarget at 0x7f8312c479

In [44]:

adata=scanpy.read_h5ad("./sobj.merged.h5ad")
barcodes=[re.sub(".df_", "", x) for x in adata.obs_names]
samplesID=[re.sub('(.)df_.+', r"\1dpf___\1dpf", x) for x in adata.obs_names]

adata.obs_names=[m+'-'+str(n) for m,n in zip(barcodes,samplesID)]

adata.obs_names
## in this step I have to make sure that I have gene_names. var: 'featuers'

Index(['AAACAGCCAACTAGGG-1-2dpf___2dpf', 'AAACAGCCATTATGGT-1-2dpf___2dpf',
       'AAACATGCATAATCGT-1-2dpf___2dpf', 'AAACCAACAACCGCCA-1-2dpf___2dpf',
       'AAACCAACAAGACTCC-1-2dpf___2dpf', 'AAACCAACACCAGGTT-1-2dpf___2dpf',
       'AAACCAACACGAATTT-1-2dpf___2dpf', 'AAACCAACATAATGAG-1-2dpf___2dpf',
       'AAACCAACATAGCTTG-1-2dpf___2dpf', 'AAACCAACATTATGCG-1-2dpf___2dpf',
       ...
       'TTTGTGAAGTCACCTC-1-6dpf___6dpf', 'TTTGTGGCAATCGCAC-1-6dpf___6dpf',
       'TTTGTGGCACATTAAC-1-6dpf___6dpf', 'TTTGTGGCAGCTCAAC-1-6dpf___6dpf',
       'TTTGTGGCAGGCCAAA-1-6dpf___6dpf', 'TTTGTGGCATAATCCG-1-6dpf___6dpf',
       'TTTGTGGCATAGCTTG-1-6dpf___6dpf', 'TTTGTGTTCACCGGTA-1-6dpf___6dpf',
       'TTTGTGTTCAGTATTG-1-6dpf___6dpf', 'TTTGTGTTCTCGACCT-1-6dpf___6dpf'],
      dtype='object', length=11923)

In [46]:
#adata.raw.to_adata().var.index=adata.var
#adata.obs.drop()

In [48]:

#adata.raw.var['gene_names']=adata.var.index
#adata.raw.var['gene_ids']=adata.var.index


In [50]:
#adata.raw.to_adata().var[['gene_names','gene_ids','_index']]

In [52]:
#cistopic_obj.cell_names=[re.sub("___.*", "", x) for x in cistopic_obj.cell_names]
#cistopic_obj.cell_names


cistopic_obj.selected_model.cell_topic

,CCTATTTAGGGATTAG-1-6dpf___6dpf,AGGAACCAGTAACCCG-1-6dpf___6dpf,GCTGGTTCAGTTTACG-1-6dpf___6dpf,AGGTTAGAGGGCCATC-1-6dpf___6dpf,GGTTTAATCCCGCATT-1-6dpf___6dpf,TTAAGGACATAGACCC-1-6dpf___6dpf,TACCGAAGTTTCGCGC-1-6dpf___6dpf,AAAGGTTAGTTAGACC-1-6dpf___6dpf,TGATTCAAGTAAGTGG-1-6dpf___6dpf,TTGTTGTTCTTAATGG-1-6dpf___6dpf,...,TTGCGTCTCTAACCAA-1-2dpf___2dpf,AGTGGACAGTATTGTG-1-2dpf___2dpf,GTGAGGAGTTGTCATC-1-2dpf___2dpf,GTAAGCAAGCTTCCCG-1-2dpf___2dpf,GGTTCTTGTCGCGCAA-1-2dpf___2dpf,GGGTTTGTCCCGTTAC-1-2dpf___2dpf,TGACCAAGTTGCACAA-1-2dpf___2dpf,CAAACGCGTGCGCGTA-1-2dpf___2dpf,CTATAACCAAGCTTAT-1-2dpf___2dpf,TATAGCCAGCAGCTAT-1-2dpf___2dpf
Topic1,0.064529,0.010851,0.014922,0.042633,0.032947,0.004888,0.022951,0.001519,0.002029,0.090038,...,0.007619,0.0075,0.041209,0.054968,0.008571,0.037594,0.040323,0.011601,0.011062,0.036176
Topic2,0.019845,0.007424,0.086387,0.012471,0.005491,0.222765,0.006557,0.009112,0.016739,0.008621,...,0.013333,0.0400,0.005495,0.029598,0.025714,0.015038,0.024194,0.041763,0.011062,0.038760
Topic3,0.048889,0.019417,0.010862,0.009281,0.041184,0.008729,0.016393,0.011390,0.002790,0.006705,...,0.011429,0.0150,0.043956,0.029598,0.008571,0.012531,0.094086,0.041763,0.035398,0.028424
Topic4,0.053489,0.016562,0.011572,0.011311,0.033557,0.004539,0.022951,0.001519,0.002029,0.047893,...,0.017143,0.1025,0.010989,0.042283,0.028571,0.007519,0.024194,0.018561,0.024336,0.062016
Topic5,0.088711,0.010280,0.019998,0.006090,0.001525,0.002793,0.006557,0.009112,0.001141,0.039272,...,0.007619,0.0300,0.021978,0.052854,0.045714,0.072682,0.034946,0.076566,0.050885,0.018088
Topic6,0.027336,0.014278,0.014821,0.007251,0.012813,0.012221,0.013115,0.010630,0.005453,0.015326,...,0.009524,0.0050,0.038462,0.016913,0.011429,0.040100,0.013441,0.046404,0.008850,0.018088
Topic7,0.023919,0.047973,0.138869,0.026972,0.037828,0.112430,0.003279,0.309795,0.006594,0.004789,...,0.222857,0.0375,0.038462,0.033827,0.020000,0.020050,0.061828,0.018561,0.015487,0.025840
Topic8,0.018136,0.004569,0.034007,0.020882,0.057657,0.001746,0.003279,0.021260,0.006848,0.003831,...,0.019048,0.0275,0.043956,0.014799,0.057143,0.010025,0.045699,0.016241,0.057522,0.043928
Topic9,0.014325,0.211308,0.022231,0.179234,0.044539,0.019553,0.003279,0.078208,0.003931,0.003831,...,0.038095,0.0350,0.024725,0.069767,0.051429,0.055138,0.026882,0.032483,0.033186,0.038760
Topic10,0.111578,0.010851,0.027104,0.013631,0.004881,0.018506,0.037705,0.006074,0.003170,0.065134,...,0.005714,0.0200,0.052198,0.021142,0.037143,0.037594,0.037634,0.018561,0.017699,0.005168


In [54]:

len(set(cistopic_obj.cell_names) & set(adata.obs_names))



7774

In [56]:
pwd

'/fs03/df22/perlaza/margo.montandon/SCENIC_MARCH2024'

In [58]:
adata.raw.to_adata().var


,_index
0,fgfr1op2
1,si:dkey-21h14.12
2,si:dkey-285e18.2
3,znf1114
4,si:dkey-21h14.10
...,...
25427,CABZ01088864.1
25428,CABZ01110379.1
25429,CABZ01109604.1
25430,ENSDARG00000101098


In [60]:
adata.var_names_make_unique()
adata

AnnData object with n_obs × n_vars = 11923 × 25432
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'nCount_ATAC', 'nFeature_ATAC', 'condition', 'nCount_SCT', 'nFeature_SCT', 'SCT.weight', 'ATAC.weight', 'wsnn_res.0.1', 'wsnn_res.0.2', 'wsnn_res.0.3', 'wsnn_res.0.4', 'wsnn_res.0.5', 'wsnn_res.0.6', 'wsnn_res.0.7', 'wsnn_res.0.8', 'wsnn_res.0.9', 'wsnn_res.1', 'wsnn_res.1.1', 'wsnn_res.1.2', 'wsnn_res.1.3', 'wsnn_res.1.4', 'wsnn_res.1.5', 'wsnn_res.1.6', 'wsnn_res.1.7', 'wsnn_res.1.8', 'wsnn_res.1.9', 'wsnn_res.2', 'seurat_clusters'
    var: 'features'
    obsm: 'X_umap.atac', 'X_umap.rna', 'X_wnn.umap'

In [62]:
# Note that if you have used the pycistarget wrapper function, the dictionary in menr.pkl 
# can be directly pass as menr in the create_SCENICPLUS_object function.


from scenicplus.scenicplus_class import create_SCENICPLUS_object
import numpy as np
scplus_obj = create_SCENICPLUS_object(
    GEX_anndata = adata ,
    cisTopic_obj = cistopic_obj,
    menr = menr,
    bc_transform_func = lambda x: x
    
)
scplus_obj.X_EXP = np.array(scplus_obj.X_EXP.todense())
scplus_obj



2024-08-06 04:19:32,441 cisTopic     INFO     Imputing region accessibility
2024-08-06 04:19:32,442 cisTopic     INFO     Impute region accessibility for regions 0-20000
2024-08-06 04:19:33,027 cisTopic     INFO     Impute region accessibility for regions 20000-40000
2024-08-06 04:19:33,622 cisTopic     INFO     Impute region accessibility for regions 40000-60000
2024-08-06 04:19:34,205 cisTopic     INFO     Impute region accessibility for regions 60000-80000
2024-08-06 04:19:34,790 cisTopic     INFO     Impute region accessibility for regions 80000-100000
2024-08-06 04:19:35,362 cisTopic     INFO     Impute region accessibility for regions 100000-120000
2024-08-06 04:19:35,918 cisTopic     INFO     Impute region accessibility for regions 120000-140000
2024-08-06 04:19:36,461 cisTopic     INFO     Impute region accessibility for regions 140000-160000
2024-08-06 04:19:37,133 cisTopic     INFO     Impute region accessibility for regions 160000-180000
2024-08-06 04:19:37,662 cisTopic     

SCENIC+ object with n_cells x n_genes = 7774 x 25432 and n_cells x n_regions = 7774 x 253880
	metadata_regions:'Chromosome', 'Start', 'End', 'Width', 'cisTopic_nr_frag', 'cisTopic_log_nr_frag', 'cisTopic_nr_acc', 'cisTopic_log_nr_acc'
	metadata_genes:'features'
	metadata_cell:'GEX_orig.ident', 'GEX_nCount_RNA', 'GEX_nFeature_RNA', 'GEX_percent.mt', 'GEX_nCount_ATAC', 'GEX_nFeature_ATAC', 'GEX_condition', 'GEX_nCount_SCT', 'GEX_nFeature_SCT', 'GEX_SCT.weight', 'GEX_ATAC.weight', 'GEX_wsnn_res.0.1', 'GEX_wsnn_res.0.2', 'GEX_wsnn_res.0.3', 'GEX_wsnn_res.0.4', 'GEX_wsnn_res.0.5', 'GEX_wsnn_res.0.6', 'GEX_wsnn_res.0.7', 'GEX_wsnn_res.0.8', 'GEX_wsnn_res.0.9', 'GEX_wsnn_res.1', 'GEX_wsnn_res.1.1', 'GEX_wsnn_res.1.2', 'GEX_wsnn_res.1.3', 'GEX_wsnn_res.1.4', 'GEX_wsnn_res.1.5', 'GEX_wsnn_res.1.6', 'GEX_wsnn_res.1.7', 'GEX_wsnn_res.1.8', 'GEX_wsnn_res.1.9', 'GEX_wsnn_res.2', 'GEX_seurat_clusters', 'ACC_SCT.weight', 'ACC_nFeature_RNA', 'ACC_Dupl_nr_frag', 'ACC_Seurat_RNA+ATAC_leiden_100_0.8', 'A

In [64]:
scplus_obj.X_EXP

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [66]:
#cluster='cluster_'

#scplus_obj.metadata_cell['ACC_Seurat_cell_type']= cluster + scplus_obj.metadata_cell['ACC_Seurat_RNA+ATAC_leiden_100_0.8'].astype(str)
#scplus_obj.metadata_cell['ACC_Seurat_cell_type']
#scplus_obj.metadata_cell['ACC_Seurat_cell_type'].value_counts()


In [68]:

#scplus_obj.subset(
 #   cells= scplus_obj.cell_names.drop( ['CGTAACCCAGCATTAT-1-2dpf___2dpf','ACCTCACTCCCTCACG-1-4dpf___4dpf']),
  #  regions=  scplus_obj.region_names, 
   # genes= scplus_obj.gene_names, 
   # return_copy=False
   # )



In [70]:
scplus_obj.metadata_cell

,GEX_orig.ident,GEX_nCount_RNA,GEX_nFeature_RNA,GEX_percent.mt,GEX_nCount_ATAC,GEX_nFeature_ATAC,GEX_condition,GEX_nCount_SCT,GEX_nFeature_SCT,GEX_SCT.weight,...,ACC_Seurat_RNA+ATAC_leiden_100_0.5,ACC_Dupl_rate,ACC_ATAC.weight,ACC_pycisTopic_leiden_10_0.6,ACC_Unique_nr_frag_in_regions,ACC_TSS_enrichment,ACC_Total_nr_frag_in_regions,ACC_barcode,ACC_wsnn_res.0.6,ACC_Seurat_RNA+ATAC_leiden_100_1.2
AGCACTAGTAAACAAG-1-2dpf___2dpf,2dpf,761.0,512,1.576873,4835.0,2445,2dpf,1696.0,525,0.488089,...,0,0.702113,0.511911,0,2050,3.847531,6944,AGCACTAGTAAACAAG-1,cluster_12,1
CTCACAACATCAGTAT-1-2dpf___2dpf,2dpf,761.0,547,2.102497,1791.0,931,2dpf,1694.0,570,0.499627,...,0,0.718535,0.500373,0,668,3.630378,2411,CTCACAACATCAGTAT-1,cluster_12,1
CAATCCCTCCGCATGA-1-6dpf___6dpf,6dpf,1933.0,890,1.551992,6171.0,2908,6dpf,1938.0,890,0.398961,...,3,0.392721,0.601039,3,1962,4.192838,3240,CAATCCCTCCGCATGA-1,cluster_1,3
GTTCGCGCAGCAAGAT-1-2dpf___2dpf,2dpf,2057.0,824,2.625182,1404.0,714,2dpf,2053.0,824,0.587619,...,0,0.714286,0.412381,0,524,3.796494,1812,GTTCGCGCAGCAAGAT-1,cluster_0,0
CTCGCTCCAGTTAAAG-1-2dpf___2dpf,2dpf,800.0,611,1.375000,1684.0,870,2dpf,1724.0,629,0.553613,...,0,0.716282,0.446387,0,629,4.309721,2239,CTCGCTCCAGTTAAAG-1,cluster_12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CTTCAATTCGGCTATG-1-2dpf___2dpf,2dpf,1320.0,758,2.575758,2713.0,1389,2dpf,1687.0,758,0.464823,...,0,0.698952,0.535177,0,1092,4.158672,3712,CTTCAATTCGGCTATG-1,cluster_0,1
GCTAGCGGTCTAACAG-1-4dpf___4dpf,4dpf,5341.0,2313,1.610185,9478.0,4151,4dpf,2655.0,1784,0.507610,...,3,0.562980,0.492390,3,2936,2.943203,6625,GCTAGCGGTCTAACAG-1,cluster_4,3
GCTTAACAGCTTCCCG-1-4dpf___4dpf,4dpf,3418.0,1520,1.433587,16491.0,7494,4dpf,2448.0,1506,0.500000,...,2,0.538859,0.500000,5,6012,5.919319,13128,GCTTAACAGCTTCCCG-1,cluster_1,14
GCTGACATCTTACTCG-1-6dpf___6dpf,6dpf,3772.0,1259,1.908802,2884.0,1415,6dpf,2455.0,1225,0.647624,...,2,0.381789,0.352376,4,878,3.007897,1398,GCTGACATCTTACTCG-1,cluster_1,2


In [72]:
scplus_obj.metadata_regions.index

Index(['17:532352-532852', '8:28901486-28901986', '13:24871987-24872487',
       '5:32591500-32592000', '23:24049830-24050330', '23:24412157-24412657',
       '15:24897736-24898236', '10:5047164-5047664', '16:27816281-27816781',
       '13:15576025-15576525',
       ...
       '8:21045354-21045854', '8:23596698-23597198', '8:3559275-3559775',
       '8:624427-624927', '8:9081262-9081762', '9:6868964-6869464',
       '9:7670341-7670841', '9:8706482-8706982', 'KZ115966.1:411-911',
       '4:34198406-34198906'],
      dtype='object', length=253880)

In [74]:
# Merge cistromes (all)
from scenicplus.cistromes import *
import time
start_time = time.time()
merge_cistromes(scplus_obj)
time = time.time()-start_time
print(time/60)

0.7673729856808981


In [76]:
import pandas as pd
genomeref = pd.read_pickle(r"GenomeRef_pyAnnotation.obj")
genomeref.df.loc
genomeref=genomeref.df.loc[:,['Chromosome', 'Start', 'End']]
#df=genomeref.df
#df['Strand']='+'
genomeref
pr_chromsizes =  pr.PyRanges(genomeref)

In [78]:
import pybiomart as pbm
import requests_cache

In [80]:

#requests_cache.install_cache('.pybiomart')
dataset = pbm.Dataset(name='drerio_gene_ensembl',  host='http://www.ensembl.org')

#dataset = pbm.Dataset(name='drerio_gene_ensembl',  host='http://asia.ensembl.org')
dataset


<biomart.Dataset name='drerio_gene_ensembl', display_name=''>

In [82]:

annot = dataset.query(attributes=['chromosome_name', 'transcription_start_site', 'strand', 'external_gene_name', 'transcript_biotype'])
annot

,Chromosome/scaffold name,Transcription start site (TSS),Strand,Gene name,Transcript type
0,24,20927202,1,fam162a,protein_coding
1,24,20927135,1,fam162a,protein_coding
2,17,50472985,-1,si:ch211-235i11.3,protein_coding
3,17,50472864,-1,si:ch211-235i11.3,protein_coding
4,1,49266886,1,caly,protein_coding
...,...,...,...,...,...
61625,11,43070883,1,CABZ01080006.1,lincRNA
61626,11,43643158,1,CABZ01073015.1,lincRNA
61627,11,44120354,-1,FO704721.3,lincRNA
61628,11,44116448,-1,FO704721.3,lincRNA


In [84]:
annot.columns=['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
annot = annot[annot.Transcript_type == 'protein_coding']
annot['Transcription_Start_Site']=annot['Start']
annot['End']=annot['Start']
annot=annot[['Chromosome', 'Start','End', 'Strand', 'Gene', 'Transcription_Start_Site','Transcript_type']]
annot=annot.loc[:,['Chromosome', 'Start','End', 'Strand', 'Gene', 'Transcription_Start_Site','Transcript_type']]


In [86]:

filter = annot['Chromosome'].str.contains('ALT')
annot = annot[~filter]
annot.Strand[annot.Strand == 1] = '+'
annot.Strand[annot.Strand == -1] = '-'
pr_annot = pr.PyRanges(annot)
pr_annot

,Chromosome,Start,End,Strand,Gene,Transcription_Start_Site,Transcript_type
0,1,49266886,49266886,+,caly,49266886,protein_coding
1,1,44949909,44949909,+,vps37c,44949909,protein_coding
2,1,44950170,44950170,+,vps37c,44950170,protein_coding
3,1,1915967,1915967,+,si:ch211-132g1.1,1915967,protein_coding
4,1,30551777,30551777,+,gpr183b,30551777,protein_coding
...,...,...,...,...,...,...,...
42044,MT,11009,11009,+,mt-nd4l,11009,protein_coding
42045,MT,11299,11299,+,mt-nd4,11299,protein_coding
42046,MT,12897,12897,+,mt-nd5,12897,protein_coding
42047,MT,15308,15308,+,mt-cyb,15308,protein_coding


In [88]:

pr_annot = pr.PyRanges(annot)
pr_annot.to_csv('pr_annotstep7.csv')
#Chromosome, Start, Strand, Gene, Transcription_Start_Site and Transcript_type

In [90]:

#pr_annot = pr.PyRanges(annot[['Chromosome', 'Start','End', 'Strand', 'Gene', 'Transcript_type','Transcription_Start_Site']])
#pr_annot

#Chromosome, Start, Strand, Gene, Transcription_Start_Site and Transcript_type, 

In [92]:
#class pr_custom(object):
#    pass

#pr_annot = pr_custom()
#pr_annot.df = annot


#class pr_custom(object):
#    pass

#pr_chromsizes = pr_custom()
#pr_chromsizes.df = genomeref

#annotation.df['Transcription_Start_Site']=annotation.df['Start']
#annotation.df['End']=annotation.df['Start']+1
#listcol=['Chromosome', 'Start', 'End','Strand', 'Gene', 'Transcript_type',
#       'Transcription_Start_Site']
#annotation.df[ listcol]

#Chromosome, Start, Strand, Gene, Transcription_Start_Site and Transcript_type
#import pyranges as pr
#pr_annot = pr.PyRanges(annotation.df[ listcol])

In [94]:
scplus_obj

#pr_annot = pr.PyRanges(annot[['Chromosome', 'Start', 'End','Gene','Transcript_type','Strand','Transcription_Start_Site']])

#pr_chromsizes = pr.PyRanges(df[['Chromosome', 'Start', 'End','Strand']])

#SCENIC+ object with n_cells x n_genes = 2415 x 21255 and n_cells x n_regions = 2415 x 191245
#        metadata_regions:'Chromosome', 'Start', 'End', 'Width', 'cisTopic_nr_frag', 'cisTopic_log_nr_frag', 'cisTopic_nr_acc', 'cisTopic_log_nr_acc'
#        metadata_genes:'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
#        metadata_cell:'GEX_n_genes', 'GEX_doublet_score', 'GEX_predicted_doublet', 'GEX_n_genes_by_counts', 'GEX_total_counts', 'GEX_total_counts_mt', 'GEX_pct_counts_mt', 'GEX_ingest_celltype_label', 'GEX_leiden_res_0.8', 'GEX_celltype', 'ACC_barcode', 'ACC_cisTopic_nr_frag', 'ACC_cisTopic_log_nr_acc', 'ACC_Total_nr_frag_in_regions', 'ACC_cisTopic_nr_acc', 'ACC_TSS_enrichment', 'ACC_Log_total_nr_frag', 'ACC_cisTopic_log_nr_frag', 'ACC_Unique_nr_frag', 'ACC_Dupl_nr_frag', 'ACC_FRIP', 'ACC_Log_unique_nr_frag', 'ACC_Unique_nr_frag_in_regions', 'ACC_Dupl_rate', 'ACC_Total_nr_frag', 'ACC_n_genes', 'ACC_doublet_score', 'ACC_predicted_doublet', 'ACC_n_genes_by_counts', 'ACC_total_counts', 'ACC_total_counts_mt', 'ACC_pct_counts_mt', 'ACC_ingest_celltype_label', 'ACC_leiden_res_0.8', 'ACC_celltype', 'ACC_sample_id'
#        menr:'CTX_topics_otsu_All', 'CTX_topics_otsu_No_promoters', 'DEM_topics_otsu_All', 'DEM_topics_otsu_No_promoters', 'CTX_topics_top_3_All', 'CTX_topics_top_3_No_promoters', 'DEM_topics_top_3_All', 'DEM_topics_top_3_No_promoters', 'CTX_DARs_All', 'CTX_DARs_No_promoters', 'DEM_DARs_All', 'DEM_DARs_No_promoters'
#        dr_cell:'GEX_X_pca', 'GEX_X_umap', 'GEX_rep'


SCENIC+ object with n_cells x n_genes = 7774 x 25432 and n_cells x n_regions = 7774 x 253880
	metadata_regions:'Chromosome', 'Start', 'End', 'Width', 'cisTopic_nr_frag', 'cisTopic_log_nr_frag', 'cisTopic_nr_acc', 'cisTopic_log_nr_acc'
	metadata_genes:'features'
	metadata_cell:'GEX_orig.ident', 'GEX_nCount_RNA', 'GEX_nFeature_RNA', 'GEX_percent.mt', 'GEX_nCount_ATAC', 'GEX_nFeature_ATAC', 'GEX_condition', 'GEX_nCount_SCT', 'GEX_nFeature_SCT', 'GEX_SCT.weight', 'GEX_ATAC.weight', 'GEX_wsnn_res.0.1', 'GEX_wsnn_res.0.2', 'GEX_wsnn_res.0.3', 'GEX_wsnn_res.0.4', 'GEX_wsnn_res.0.5', 'GEX_wsnn_res.0.6', 'GEX_wsnn_res.0.7', 'GEX_wsnn_res.0.8', 'GEX_wsnn_res.0.9', 'GEX_wsnn_res.1', 'GEX_wsnn_res.1.1', 'GEX_wsnn_res.1.2', 'GEX_wsnn_res.1.3', 'GEX_wsnn_res.1.4', 'GEX_wsnn_res.1.5', 'GEX_wsnn_res.1.6', 'GEX_wsnn_res.1.7', 'GEX_wsnn_res.1.8', 'GEX_wsnn_res.1.9', 'GEX_wsnn_res.2', 'GEX_seurat_clusters', 'ACC_SCT.weight', 'ACC_nFeature_RNA', 'ACC_Dupl_nr_frag', 'ACC_Seurat_RNA+ATAC_leiden_100_0.8', 'A

In [96]:
type(pr_annot.chromosomes)


list

In [98]:

str(pr_chromsizes.chromosomes)

"['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', 'KN147632.2', 'KN147636.1', 'KN147637.2', 'KN147651.2', 'KN149680.1', 'KN149682.1', 'KN149684.1', 'KN149685.1', 'KN149686.1', 'KN149688.2', 'KN149689.2', 'KN149690.1', 'KN149695.1', 'KN149696.2', 'KN149697.1', 'KN149698.1', 'KN149702.1', 'KN149707.2', 'KN149710.1', 'KN149711.1', 'KN149713.1', 'KN149715.1', 'KN149725.1', 'KN149729.1', 'KN149732.1', 'KN149735.1', 'KN149739.1', 'KN149749.1', 'KN149764.1', 'KN149778.1', 'KN149779.1', 'KN149782.1', 'KN149790.1', 'KN149793.1', 'KN149797.1', 'KN149800.1', 'KN149803.1', 'KN149813.1', 'KN149816.1', 'KN149818.1', 'KN149840.1', 'KN149843.1', 'KN149847.1', 'KN149855.1', 'KN149857.1', 'KN149859.1', 'KN149874.1', 'KN149880.1', 'KN149883.1', 'KN149884.1', 'KN149895.1', 'KN149897.1', 'KN149909.1', 'KN149912.1', 'KN149914.1', 'KN149916.1', 'KN149932.1', 'KN149936.1', 'KN149943.1', 'KN149946.1', 'KN149948.1', 'KN

In [100]:
chromosome_list = pd.DataFrame(
    {'chromsize': pr_chromsizes.chromosomes,
     'annot': pr_annot.chromosomes,
    })
chromosome_list


,chromsize,annot
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
261,KZ116053.1,KZ116053.1
262,KZ116055.1,KZ116055.1
263,KZ116064.1,KZ116064.1
264,KZ116065.1,KZ116065.1


In [101]:
scplus_obj.metadata_regions['Chromosome'].tolist


<bound method IndexOpsMixin.tolist of 17:532352-532852                17
8:28901486-28901986              8
13:24871987-24872487            13
5:32591500-32592000              5
23:24049830-24050330            23
                           ...    
9:6868964-6869464                9
9:7670341-7670841                9
9:8706482-8706982                9
KZ115966.1:411-911      KZ115966.1
4:34198406-34198906              4
Name: Chromosome, Length: 253880, dtype: object>

In [102]:
pr_chromsizes

,Chromosome,Start,End
0,1,0,43039672
1,2,0,39555667
2,3,0,60282382
3,4,0,76270779
4,5,0,61329656
...,...,...,...
262,KZ116053.1,0,10784
263,KZ116055.1,0,1335
264,KZ116064.1,0,82859
265,KZ116065.1,0,20619


In [103]:
pr_annot.copy

<bound method PyRanges.copy of +--------------+-----------+-----------+--------------+-------+
| Chromosome   | Start     | End       | Strand       | +3    |
| (category)   | (int64)   | (int64)   | (category)   | ...   |
|--------------+-----------+-----------+--------------+-------|
| 1            | 49266886  | 49266886  | +            | ...   |
| 1            | 44949909  | 44949909  | +            | ...   |
| 1            | 44950170  | 44950170  | +            | ...   |
| 1            | 1915967   | 1915967   | +            | ...   |
| ...          | ...       | ...       | ...          | ...   |
| MT           | 11299     | 11299     | +            | ...   |
| MT           | 12897     | 12897     | +            | ...   |
| MT           | 15308     | 15308     | +            | ...   |
| MT           | 15232     | 15232     | -            | ...   |
+--------------+-----------+-----------+--------------+-------+
Stranded PyRanges object has 42,049 rows and 7 columns from 266 chromosom

In [104]:
pr_chromsizes.copy

<bound method PyRanges.copy of +--------------+-----------+-----------+
| Chromosome   | Start     | End       |
| (category)   | (int64)   | (int64)   |
|--------------+-----------+-----------|
| 1            | 0         | 43039672  |
| 2            | 0         | 39555667  |
| 3            | 0         | 60282382  |
| 4            | 0         | 76270779  |
| ...          | ...       | ...       |
| KZ116055.1   | 0         | 1335      |
| KZ116064.1   | 0         | 82859     |
| KZ116065.1   | 0         | 20619     |
| MT           | 0         | 14717     |
+--------------+-----------+-----------+
Unstranded PyRanges object has 267 rows and 3 columns from 266 chromosomes.
For printing, the PyRanges was sorted on Chromosome.>

In [105]:
scplus_obj.metadata_genes['features']

fgfr1op2                        fgfr1op2
si:dkey-21h14.12        si:dkey-21h14.12
si:dkey-285e18.2        si:dkey-285e18.2
znf1114                          znf1114
si:dkey-21h14.10        si:dkey-21h14.10
                             ...        
CABZ01088864.1            CABZ01088864.1
CABZ01110379.1            CABZ01110379.1
CABZ01109604.1            CABZ01109604.1
ENSDARG00000101098    ENSDARG00000101098
ENSDARG00000103574    ENSDARG00000103574
Name: features, Length: 25432, dtype: object

In [106]:
#because I have custom annotation and chromosome sizes I need to run get_search_space outside of the wrapper function (run_scenicplus)
from typing import Optional
from scenicplus.enhancer_to_gene import *






In [107]:

get_search_space(
    scplus_obj,
    upstream = [100, 15000], 
    downstream = [100, 15000],
    pr_annot = pr_annot, #see above
    pr_chromsizes = pr_chromsizes, #see above
    biomart_host = 'http://www.ensembl.org')

2024-08-06 04:25:15,033 R2G          INFO     Extending promoter annotation to 10 bp upstream and 10 downstream
Warning! Start and End columns now have different dtypes: int32 and int64
Warning! Start and End columns now have different dtypes: int32 and int64
2024-08-06 04:25:15,954 R2G          INFO     Extending search space to:
            						15000 bp downstream of the end of the gene.
            						15000 bp upstream of the start of the gene.
Warning! Start and End columns now have different dtypes: int32 and int64
Warning! Start and End columns now have different dtypes: int32 and int64
2024-08-06 04:25:22,698 R2G          INFO     Intersecting with regions.


join: Strand data from other will be added as strand data to self.
If this is undesired use the flag apply_strand_suffix=False.
To turn off the warning set apply_strand_suffix to True or False.


Warning! Start and End columns now have different dtypes: int32 and int64
2024-08-06 04:25:24,025 R2G          INFO     Calculating distances from region to gene
2024-08-06 04:25:47,613 R2G          INFO     Imploding multiple entries per region and gene
2024-08-06 04:26:09,084 R2G          INFO     Done!


In [108]:
# the tf_file
#wget http://bioinfo.life.hust.edu.cn/static/AnimalTFDB3/download/Danio_rerio_TF
#cut -f2 Danio_rerio_TF >Danio_rerio_TF_listgenes.txt

#mkdir drerio_bedToBigBed
#wget -O drerio_bedToBigBed/bedToBigBed http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/bedToBigBed
#chmod +x drerio_bedToBigBed/bedToBigBed



In [109]:
scplus_obj.metadata_cell

,GEX_orig.ident,GEX_nCount_RNA,GEX_nFeature_RNA,GEX_percent.mt,GEX_nCount_ATAC,GEX_nFeature_ATAC,GEX_condition,GEX_nCount_SCT,GEX_nFeature_SCT,GEX_SCT.weight,...,ACC_Seurat_RNA+ATAC_leiden_100_0.5,ACC_Dupl_rate,ACC_ATAC.weight,ACC_pycisTopic_leiden_10_0.6,ACC_Unique_nr_frag_in_regions,ACC_TSS_enrichment,ACC_Total_nr_frag_in_regions,ACC_barcode,ACC_wsnn_res.0.6,ACC_Seurat_RNA+ATAC_leiden_100_1.2
AGCACTAGTAAACAAG-1-2dpf___2dpf,2dpf,761.0,512,1.576873,4835.0,2445,2dpf,1696.0,525,0.488089,...,0,0.702113,0.511911,0,2050,3.847531,6944,AGCACTAGTAAACAAG-1,cluster_12,1
CTCACAACATCAGTAT-1-2dpf___2dpf,2dpf,761.0,547,2.102497,1791.0,931,2dpf,1694.0,570,0.499627,...,0,0.718535,0.500373,0,668,3.630378,2411,CTCACAACATCAGTAT-1,cluster_12,1
CAATCCCTCCGCATGA-1-6dpf___6dpf,6dpf,1933.0,890,1.551992,6171.0,2908,6dpf,1938.0,890,0.398961,...,3,0.392721,0.601039,3,1962,4.192838,3240,CAATCCCTCCGCATGA-1,cluster_1,3
GTTCGCGCAGCAAGAT-1-2dpf___2dpf,2dpf,2057.0,824,2.625182,1404.0,714,2dpf,2053.0,824,0.587619,...,0,0.714286,0.412381,0,524,3.796494,1812,GTTCGCGCAGCAAGAT-1,cluster_0,0
CTCGCTCCAGTTAAAG-1-2dpf___2dpf,2dpf,800.0,611,1.375000,1684.0,870,2dpf,1724.0,629,0.553613,...,0,0.716282,0.446387,0,629,4.309721,2239,CTCGCTCCAGTTAAAG-1,cluster_12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CTTCAATTCGGCTATG-1-2dpf___2dpf,2dpf,1320.0,758,2.575758,2713.0,1389,2dpf,1687.0,758,0.464823,...,0,0.698952,0.535177,0,1092,4.158672,3712,CTTCAATTCGGCTATG-1,cluster_0,1
GCTAGCGGTCTAACAG-1-4dpf___4dpf,4dpf,5341.0,2313,1.610185,9478.0,4151,4dpf,2655.0,1784,0.507610,...,3,0.562980,0.492390,3,2936,2.943203,6625,GCTAGCGGTCTAACAG-1,cluster_4,3
GCTTAACAGCTTCCCG-1-4dpf___4dpf,4dpf,3418.0,1520,1.433587,16491.0,7494,4dpf,2448.0,1506,0.500000,...,2,0.538859,0.500000,5,6012,5.919319,13128,GCTTAACAGCTTCCCG-1,cluster_1,14
GCTGACATCTTACTCG-1-6dpf___6dpf,6dpf,3772.0,1259,1.908802,2884.0,1415,6dpf,2455.0,1225,0.647624,...,2,0.381789,0.352376,4,878,3.007897,1398,GCTGACATCTTACTCG-1,cluster_1,2


In [110]:
#scplus_obj.metadata_cell['GEX_wsnn_res.0.8']

scplus_obj.metadata_cell['GEX_celltype']

scplus_obj.metadata_cell['GEX_celltype'].value_counts()





KeyError: 'GEX_celltype'

In [111]:
scplus_obj.metadata_cell['GEX_celltype']=scplus_obj.metadata_cell['ACC_wsnn_res.0.1'].astype(str)
scplus_obj.metadata_cell['GEX_celltype']

AGCACTAGTAAACAAG-1-2dpf___2dpf    cluster_4
CTCACAACATCAGTAT-1-2dpf___2dpf    cluster_4
CAATCCCTCCGCATGA-1-6dpf___6dpf    cluster_0
GTTCGCGCAGCAAGAT-1-2dpf___2dpf    cluster_1
CTCGCTCCAGTTAAAG-1-2dpf___2dpf    cluster_4
                                    ...    
CTTCAATTCGGCTATG-1-2dpf___2dpf    cluster_1
GCTAGCGGTCTAACAG-1-4dpf___4dpf    cluster_0
GCTTAACAGCTTCCCG-1-4dpf___4dpf    cluster_0
GCTGACATCTTACTCG-1-6dpf___6dpf    cluster_0
TCTAAGGGTCACCTAT-1-4dpf___4dpf    cluster_4
Name: GEX_celltype, Length: 7774, dtype: object

In [112]:
scplus_obj.metadata_cell['GEX_celltype'].value_counts()


cluster_0     2957
cluster_1     1616
cluster_4     1001
cluster_6      576
cluster_7      405
cluster_8      390
cluster_9      267
cluster_10     165
cluster_11     155
cluster_2      121
cluster_3      101
cluster_5       20
Name: GEX_celltype, dtype: int64

In [113]:
scplus_obj

SCENIC+ object with n_cells x n_genes = 7774 x 25432 and n_cells x n_regions = 7774 x 253880
	metadata_regions:'Chromosome', 'Start', 'End', 'Width', 'cisTopic_nr_frag', 'cisTopic_log_nr_frag', 'cisTopic_nr_acc', 'cisTopic_log_nr_acc'
	metadata_genes:'features'
	metadata_cell:'GEX_orig.ident', 'GEX_nCount_RNA', 'GEX_nFeature_RNA', 'GEX_percent.mt', 'GEX_nCount_ATAC', 'GEX_nFeature_ATAC', 'GEX_condition', 'GEX_nCount_SCT', 'GEX_nFeature_SCT', 'GEX_SCT.weight', 'GEX_ATAC.weight', 'GEX_wsnn_res.0.1', 'GEX_wsnn_res.0.2', 'GEX_wsnn_res.0.3', 'GEX_wsnn_res.0.4', 'GEX_wsnn_res.0.5', 'GEX_wsnn_res.0.6', 'GEX_wsnn_res.0.7', 'GEX_wsnn_res.0.8', 'GEX_wsnn_res.0.9', 'GEX_wsnn_res.1', 'GEX_wsnn_res.1.1', 'GEX_wsnn_res.1.2', 'GEX_wsnn_res.1.3', 'GEX_wsnn_res.1.4', 'GEX_wsnn_res.1.5', 'GEX_wsnn_res.1.6', 'GEX_wsnn_res.1.7', 'GEX_wsnn_res.1.8', 'GEX_wsnn_res.1.9', 'GEX_wsnn_res.2', 'GEX_seurat_clusters', 'ACC_SCT.weight', 'ACC_nFeature_RNA', 'ACC_Dupl_nr_frag', 'ACC_Seurat_RNA+ATAC_leiden_100_0.8', 'A

In [114]:
#RAY_IGNORE_UNHANDLED_ERRORS=1
from typing import Optional
#from scenicplus.wrappers import *
from scenicplus.wrappers.run_scenicplus import *



In [115]:
#import importlib

# make changes to example.py file

scplus_obj = dill.load(open(os.path.join(outDir, 'scenicplus/scplus_obj.pkl'), 'rb'))



#ray.shutdown()
try:
    run_scenicplus(
        scplus_obj = scplus_obj,
        variable = ['GEX_celltype'],
        species = 'custom',
        assembly = 'custom',
        tf_file = 'Danio_rerio_TF_listgenes.txt',
        save_path = os.path.join(outDir, 'scenicplus'),
        biomart_host = 'http://www.ensembl.org',
        upstream = [1000, 150000],
        downstream = [1000, 150000],
        calculate_TF_eGRN_correlation = False,
        calculate_DEGs_DARs = True,
        export_to_loom_file = True,
        export_to_UCSC_file = False,
        n_cpu = 30,
        _temp_dir = os.path.join(tmpDir, 'ray_spill'))
except Exception as e:
    #in case of failure, still save the object
    dill.dump(scplus_obj, open(os.path.join(outDir, 'scenicplus/scplus_obj.pkl'), 'wb'), protocol=-1)
    raise(e)

2024-08-06 04:28:04,331 SCENIC+_wrapper INFO     SCENIC_results_MARCH2024/scenicplus folder already exists.
2024-08-06 04:28:04,333 SCENIC+_wrapper INFO     Calculating DEGs/DARs
2024-08-06 04:28:04,333 SCENIC+      INFO     Calculating DEGs for variable GEX_celltype
2024-08-06 04:28:07,350 SCENIC+      INFO     There are 4377 variable features


... storing 'GEX_orig.ident' as categorical
... storing 'GEX_condition' as categorical
... storing 'ACC_orig.ident' as categorical
... storing 'ACC_wsnn_res.0.2' as categorical
... storing 'ACC_wsnn_res.0.8' as categorical
... storing 'ACC_clusters' as categorical
... storing 'ACC_wsnn_res.1.6' as categorical
... storing 'ACC_wsnn_res.0.9' as categorical
... storing 'ACC_wsnn_res.1.9' as categorical
... storing 'ACC_wsnn_res.1.7' as categorical
... storing 'ACC_wsnn_res.0.4' as categorical
... storing 'ACC_wsnn_res.1.5' as categorical
... storing 'ACC_seurat_clusters' as categorical
... storing 'ACC_barcodes' as categorical
... storing 'ACC_wsnn_res.1.8' as categorical
... storing 'ACC_wsnn_res.1.2' as categorical
... storing 'ACC_wsnn_res.1.1' as categorical
... storing 'ACC_sample_id' as categorical
... storing 'ACC_condition' as categorical
... storing 'ACC_wsnn_res.0.1' as categorical
... storing 'ACC_wsnn_res.0.7' as categorical
... storing 'ACC_wsnn_res.1' as categorical
... stor

2024-08-06 04:28:11,561 SCENIC+      INFO     Finished calculating DEGs for variable GEX_celltype
2024-08-06 04:28:11,562 SCENIC+      INFO     Calculating DARs for variable GEX_celltype
2024-08-06 04:28:57,648 SCENIC+      INFO     There are 66915 variable features


... storing 'GEX_orig.ident' as categorical
... storing 'GEX_condition' as categorical
... storing 'ACC_orig.ident' as categorical
... storing 'ACC_wsnn_res.0.2' as categorical
... storing 'ACC_wsnn_res.0.8' as categorical
... storing 'ACC_clusters' as categorical
... storing 'ACC_wsnn_res.1.6' as categorical
... storing 'ACC_wsnn_res.0.9' as categorical
... storing 'ACC_wsnn_res.1.9' as categorical
... storing 'ACC_wsnn_res.1.7' as categorical
... storing 'ACC_wsnn_res.0.4' as categorical
... storing 'ACC_wsnn_res.1.5' as categorical
... storing 'ACC_seurat_clusters' as categorical
... storing 'ACC_barcodes' as categorical
... storing 'ACC_wsnn_res.1.8' as categorical
... storing 'ACC_wsnn_res.1.2' as categorical
... storing 'ACC_wsnn_res.1.1' as categorical
... storing 'ACC_sample_id' as categorical
... storing 'ACC_condition' as categorical
... storing 'ACC_wsnn_res.0.1' as categorical
... storing 'ACC_wsnn_res.0.7' as categorical
... storing 'ACC_wsnn_res.1' as categorical
... stor

2024-08-06 04:30:14,292 SCENIC+      INFO     Finished calculating DARs for variable GEX_celltype
2024-08-06 04:30:14,294 SCENIC+_wrapper INFO     Exporting to loom file
2024-08-06 04:30:14,295 SCENIC+      INFO     Formatting data
2024-08-06 04:30:14,822 SCENIC+      INFO     Creating minimal loom
2024-08-06 04:30:21,337 SCENIC+      INFO     Adding annotations
2024-08-06 04:30:23,394 SCENIC+      INFO     Adding clusterings
2024-08-06 04:30:23,483 SCENIC+      INFO     Adding markers
2024-08-06 04:30:23,857 SCENIC+      INFO     Exporting
2024-08-06 04:30:28,325 SCENIC+      INFO     Formatting data
2024-08-06 04:31:00,588 SCENIC+      INFO     Creating minimal loom
2024-08-06 04:33:06,111 SCENIC+      INFO     Adding annotations
2024-08-06 04:33:40,350 SCENIC+      INFO     Adding clusterings
2024-08-06 04:33:40,446 SCENIC+      INFO     Adding markers
2024-08-06 04:33:44,517 SCENIC+      INFO     Exporting
2024-08-06 04:35:17,241 SCENIC+_wrapper INFO     Saving object
2024-08-06 04

In [116]:
scplus_obj


SCENIC+ object with n_cells x n_genes = 7774 x 25432 and n_cells x n_regions = 7774 x 253880
	metadata_regions:'Chromosome', 'Start', 'End', 'Width', 'cisTopic_nr_frag', 'cisTopic_log_nr_frag', 'cisTopic_nr_acc', 'cisTopic_log_nr_acc'
	metadata_genes:'features'
	metadata_cell:'GEX_orig.ident', 'GEX_nCount_RNA', 'GEX_nFeature_RNA', 'GEX_percent.mt', 'GEX_nCount_ATAC', 'GEX_nFeature_ATAC', 'GEX_condition', 'GEX_nCount_SCT', 'GEX_nFeature_SCT', 'GEX_SCT.weight', 'GEX_ATAC.weight', 'GEX_wsnn_res.0.1', 'GEX_wsnn_res.0.2', 'GEX_wsnn_res.0.3', 'GEX_wsnn_res.0.4', 'GEX_wsnn_res.0.5', 'GEX_wsnn_res.0.6', 'GEX_wsnn_res.0.7', 'GEX_wsnn_res.0.8', 'GEX_wsnn_res.0.9', 'GEX_wsnn_res.1', 'GEX_wsnn_res.1.1', 'GEX_wsnn_res.1.2', 'GEX_wsnn_res.1.3', 'GEX_wsnn_res.1.4', 'GEX_wsnn_res.1.5', 'GEX_wsnn_res.1.6', 'GEX_wsnn_res.1.7', 'GEX_wsnn_res.1.8', 'GEX_wsnn_res.1.9', 'GEX_wsnn_res.2', 'GEX_seurat_clusters', 'ACC_SCT.weight', 'ACC_nFeature_RNA', 'ACC_Dupl_nr_frag', 'ACC_Seurat_RNA+ATAC_leiden_100_0.8', 'A

scplus_obj.uns['region_to_gene']

In [117]:
scplus_obj.uns['TF2G_adj']

,TF,target,importance,regulation,rho,importance_x_rho,importance_x_abs_rho
0,AL627305.1,cyp46a1.3,4.575576e-02,1,0.063169,2.890347e-03,2.890347e-03
1,AL627305.1,ftr86,2.493353e-02,1,0.051732,1.289870e-03,1.289870e-03
2,AL627305.1,znf414,6.883564e-07,0,-0.000683,-4.704680e-10,4.704680e-10
3,AL627305.1,srpk1a,2.692435e-01,1,0.102912,2.770827e-02,2.770827e-02
4,AL627305.1,slc2a11b,4.863833e-10,1,0.034289,1.667765e-11,1.667765e-11
...,...,...,...,...,...,...,...
1937361,zbtb22b,zbtb22b,3.921423e+00,1,1.000000,3.921423e+00,3.921423e+00
1937362,esr2a,esr2a,2.407375e+00,1,1.000000,2.407375e+00,2.407375e+00
1937363,znf143b,znf143b,5.966119e+00,1,1.000000,5.966119e+00,5.966119e+00
1937364,mafb,mafb,2.133625e+00,1,1.000000,2.133625e+00,2.133625e+00


In [118]:
scplus_obj.uns['eRegulons'][0:5]

[eRegulon for TF atf3 in context frozenset({'0.85 quantile', 'Top 10 region-to-gene links per gene', 'positive tf2g', 'Top 5 region-to-gene links per gene', 'BASC binarized', 'Top 15 region-to-gene links per gene', '0.95 quantile', '0.9 quantile', 'Cistromes_Unfiltered', 'positive r2g'}).
 	This eRegulon has 472 target regions and 294 target genes.,
 eRegulon for TF atf4a in context frozenset({'Top 10 region-to-gene links per gene', 'positive tf2g', 'Top 5 region-to-gene links per gene', 'Top 15 region-to-gene links per gene', 'Cistromes_Unfiltered', 'positive r2g'}).
 	This eRegulon has 150 target regions and 132 target genes.,
 eRegulon for TF atf4b in context frozenset({'positive r2g', 'positive tf2g', 'Top 15 region-to-gene links per gene', 'Cistromes_Unfiltered'}).
 	This eRegulon has 177 target regions and 145 target genes.,
 eRegulon for TF atf6 in context frozenset({'Top 10 region-to-gene links per gene', 'positive tf2g', 'Top 5 region-to-gene links per gene', 'BASC binarized',

In [119]:
from scenicplus.utils import format_egrns
from scenicplus.eregulon_enrichment import get_eRegulons_as_signatures
format_egrns(scplus_obj)
get_eRegulons_as_signatures(scplus_obj)

In [120]:
scplus_obj.uns['eRegulon_metadata']


,Region_signature_name,Gene_signature_name,TF,is_extended,Region,Gene,R2G_importance,R2G_rho,R2G_importance_x_rho,R2G_importance_x_abs_rho,TF2G_importance,TF2G_regulation,TF2G_rho,TF2G_importance_x_abs_rho,TF2G_importance_x_rho
0,atf3_+_+_(472r),atf3_+_+_(294g),atf3,False,2:2102148-2102648,dspa,0.086590,0.094114,0.008149,0.008149,0.801664,1,0.358841,0.287670,0.287670
1,atf3_+_+_(472r),atf3_+_+_(294g),atf3,False,2:2107207-2107707,dspa,0.099698,0.100741,0.010044,0.010044,0.801664,1,0.358841,0.287670,0.287670
2,atf3_+_+_(472r),atf3_+_+_(294g),atf3,False,2:2104810-2105310,dspa,0.086578,0.090196,0.007809,0.007809,0.801664,1,0.358841,0.287670,0.287670
3,atf3_+_+_(472r),atf3_+_+_(294g),atf3,False,2:2102667-2103167,dspa,0.067521,0.086909,0.005868,0.005868,0.801664,1,0.358841,0.287670,0.287670
4,atf3_+_+_(472r),atf3_+_+_(294g),atf3,False,14:30907734-30908234,foxo4,0.099241,0.109847,0.010901,0.010901,0.454781,1,0.279075,0.126918,0.126918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,zeb1b_extended_+_-_(12r),zeb1b_extended_+_-_(11g),zeb1b,True,3:32476577-32477077,scaf1,0.061195,-0.056674,-0.003468,0.003468,0.535870,1,0.207893,0.111403,0.111403
8,zeb1b_extended_+_-_(12r),zeb1b_extended_+_-_(11g),zeb1b,True,21:5924493-5924993,rexo4,0.054471,-0.102886,-0.005604,0.005604,2.182333,1,0.241762,0.527605,0.527605
9,zeb1b_extended_+_-_(12r),zeb1b_extended_+_-_(11g),zeb1b,True,3:13196677-13197177,sun1,0.018787,-0.061842,-0.001162,0.001162,0.899646,1,0.277533,0.249681,0.249681
10,zeb1b_extended_+_-_(12r),zeb1b_extended_+_-_(11g),zeb1b,True,4:16884310-16884810,tmpoa,0.075031,-0.099090,-0.007435,0.007435,0.579173,1,0.110439,0.063963,0.063963


In [121]:
type(scplus_obj.uns['eRegulon_metadata'])

pandas.core.frame.DataFrame

In [122]:
filenames_erg=os.path.join(outDir,'eRegulon_metadata_allspecies.csv')

In [123]:
filenames_erg

'SCENIC_results_MARCH2024/eRegulon_metadata_allspecies.csv'

In [124]:
scplus_obj.uns['eRegulon_metadata'].to_csv(r'SCENIC_results_MARCH2024/eRegulon_metadata_allspecies.csv')

In [127]:
# combine variables SCENIC+_leiden_10_0.6 and sample
scplus_obj.metadata_cell["sample_Res.0.6"] = scplus_obj.metadata_cell['SCENIC+_leiden_10_0.6'] + "_" + scplus_obj.metadata_cell['GEX_orig.ident']
scplus_obj.metadata_cell["sample_Res.0.8"] = scplus_obj.metadata_cell['SCENIC+_leiden_10_0.8'] + "_" + scplus_obj.metadata_cell['GEX_orig.ident']
scplus_obj.metadata_cell["sample_Res.1.0"] = scplus_obj.metadata_cell['SCENIC+_leiden_10_1.0'] + "_" + scplus_obj.metadata_cell['GEX_orig.ident']
scplus_obj.metadata_cell["sample_Res.1.2"] = scplus_obj.metadata_cell['SCENIC+_leiden_10_1.2'] + "_" + scplus_obj.metadata_cell['GEX_orig.ident']



KeyError: 'SCENIC+_leiden_10_0.6'

In [126]:

scplus_obj = dill.load(open(os.path.join(outDir, 'scenicplus/scplus_obj.pkl'), 'rb'))



2024-08-06 04:55:48,454 SCENIC+_wrapper INFO     SCENIC_results_MARCH2024/scenicplus folder already exists.
2024-08-06 04:55:48,457 SCENIC+_wrapper INFO     Calculating DEGs/DARs
2024-08-06 04:55:48,458 SCENIC+      INFO     Calculating DEGs for variable sample_Res.0.6


KeyError: 'sample_Res.0.6'